In [ ]:
import jax.numpy as jnp
import jax
#jax.config.update("jax_debug_nans", True)
#jax.config.update('jax_disable_jit', True)

import matplotlib.pyplot as plt
import time
from PIL import Image, ImageDraw


In [ ]:
image_width, image_height = 200, 100
image = Image.new('L', (image_width, image_height))
draw = ImageDraw.Draw(image)

draw.circle([20, 50], 10, outline="white")
draw.circle([100, 20], 5, outline="white")

im = jnp.array(image)

def gaussian_blur(image, kernel_size=3, sigma=1.0):
    """Applies a Gaussian blur to the input image."""
    def gaussian_kernel(size, sigma):
        """Generates a Gaussian kernel."""
        ax = jnp.arange(-size // 2 + 1., size // 2 + 1.)
        xx, yy = jnp.meshgrid(ax, ax)
        kernel = jnp.exp(-0.5 * (jnp.square(xx) + jnp.square(yy)) / jnp.square(sigma))
        return kernel / jnp.sum(kernel)
    
    kernel = gaussian_kernel(kernel_size, sigma)
    return jax.scipy.signal.convolve2d(image, kernel, mode='same')

im_blurred = gaussian_blur(im)

plt.imshow(im_blurred, origin='lower')


In [ ]:
def smooth_abs(x):
    return x * jnp.tanh(10.0 * x)

def softplus(x):
    return jax.nn.softplus(50 * x) / 50

def softminus(x):
    return x - softplus(x)

def box_unsigned(x, y, width, height):
    q_x = softplus(smooth_abs(x) - width / 2)
    q_y = softplus(smooth_abs(y) - height / 2)
    return jnp.sqrt(q_x*q_x + q_y*q_y) + softminus(jnp.maximum(q_x, q_y))


def box(x, y, width, height):
    # ref https://iquilezles.org/articles/distfunctions/
    p = jnp.array([x,y])
    b = jnp.array([width/2, height/2])
    q = smooth_abs(p) - b
    return jnp.linalg.norm(jnp.maximum(q, 0.0)) + softminus(jnp.max(q))

def circle(x, y, radius):
    p = jnp.array([x,y])
    return jnp.linalg.norm(p) - radius

def distance_to_rectangle(x, y, rect_x, rect_y, rect_w, rect_h):
    return box(x - rect_x, y - rect_y, rect_w, rect_h)

def distance_to_circle(x, y, circle_x, circle_y, circle_r):
    return circle(x - circle_x, y - circle_y, circle_r)


def signed_distance_field(params, image_width, image_height):

    x_coords = jnp.arange(image_width)
    y_coords = jnp.arange(image_height)
    xx, yy = jnp.meshgrid(x_coords, y_coords)

    # rect_x, rect_y, rect_w, rect_h = params
    # sdf = jax.vmap(jax.vmap(lambda x, y: distance_to_rectangle(x, y, rect_x, rect_y, rect_w, rect_h)))(yy, xx)    

    circle_x, circle_y, circle_r = params
    sdf = jax.vmap(jax.vmap(lambda x, y: distance_to_circle(x, y, circle_x, circle_y, circle_r)))(yy, xx)    
    return sdf


params  = jnp.array([50.1, 20.1, 20.1])
sdf = signed_distance_field(params, image_width, image_height)
plt.figure(figsize=(6, 6))
plt.imshow(sdf, cmap='BrBG', origin='lower', vmin=-jnp.max(jnp.abs(sdf)), vmax=jnp.max(jnp.abs(sdf)))
plt.colorbar(label='Signed Distance')
plt.contour(sdf, levels=[0], colors='black')
plt.title('Signed Distance Field')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [ ]:
def plot_circle_on_image(image, circles):
    fig, ax = plt.subplots()
    ax.imshow(image, cmap='gray', origin='lower')
    colors = ['r', 'g', 'b', 'y', 'c']  # Define up to 5 colors
    for i, (cx, cy, r) in enumerate(circles):
        color = colors[i % len(colors)]  # Cycle through colors
        circle = plt.Circle((cx, cy), r, color=color, fill=False)
        ax.add_patch(circle)
    plt.show()

In [ ]:
import optax
import optax.tree_utils as otu

from functools import partial
_lbfgs = optax.lbfgs()


@partial(jax.jit, static_argnames=["fun"])
def _run_lbfgs(init_params, lbfgs_state, fun, max_steps, tolerance, **kwargs):
    value_and_grad_fun = optax.value_and_grad_from_state(fun)
    
    def step(carry):
        params, state = carry
        value, grad = value_and_grad_fun(params, **kwargs, state=state)
        #jax.debug.print("value: {value}, grad: {grad}, params: {params}", value=value, grad=grad, params=params)

        updates, state = _lbfgs.update(
            grad, state, params, **kwargs, value=value, grad=grad, value_fn=fun
        )
        params = optax.apply_updates(params, updates)

        return params, state

    def continuing_criterion(carry):
        _, state = carry
        iter_num = otu.tree_get(state, "count")
        grad = otu.tree_get(state, "grad")
        err = otu.tree_l2_norm(grad)
        return (iter_num == 0) | ((iter_num < max_steps) & (err >= tolerance))

    if lbfgs_state is None:
        state = _lbfgs.init(init_params)
    else:
        # reset iteration count leftover from previous solve
        state = otu.tree_set(lbfgs_state, count=0)


    init_carry = (init_params, state)
    final_params, final_state = jax.lax.while_loop(
        continuing_criterion, step, init_carry
    )
    return final_params, final_state


def solve(params, loss_fn, target_image):
    nonzero_points = jnp.argwhere(target_image)

    start_time = time.time()
    soln, state = _run_lbfgs(
        params,
        None, # minimizer state
        loss_fn,
        max_steps=1000,
        tolerance=1e-3,
        nonzero_points=nonzero_points,
    )
    end_time = time.time()
    iter_num = otu.tree_get(state, "count")
    print(f"Elapsed time: {end_time - start_time:.5f} seconds; {iter_num} iterations")

    return soln

In [ ]:
@jax.jit
def loss_fn_circle(params, nonzero_points):
    def distance_to_circle(x, y, circle_x, circle_y, circle_r):
        return circle(x - circle_x, y - circle_y, circle_r)
    
    loss = jax.vmap(lambda point: distance_to_circle(point[1], point[0], *params[0:3]))(nonzero_points)**2.0
    return loss.sum()

In [ ]:
solution = solve(jnp.array([10., 10, 5]), loss_fn_circle, im_blurred)
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))

In [ ]:
@jax.jit
def loss_fn_circle(params, nonzero_points):
    def distance_to_circle(x, y, circle_x, circle_y, circle_r):
        return circle(x - circle_x, y - circle_y, circle_r)
    
    loss = jax.vmap(lambda point: distance_to_circle(point[1], point[0], *params[0:3]))(nonzero_points)**2.0
    return loss.sum()

In [ ]:
solution = solve(jnp.array([10., 10, 5]), loss_fn_circle, im_blurred)
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))

In [ ]:
@jax.jit
def loss_fn_two_circles(params, nonzero_points):
    loss = jax.vmap(lambda idx: jnp.minimum(distance_to_circle(idx[1], idx[0], *params[0:3]), distance_to_circle(idx[1], idx[0], *params[3:6]))**2)(nonzero_points)    
    return loss.sum()

In [ ]:
params = jnp.array([10., 10, 5, 150, 20, 5])
solution = solve(params, loss_fn_two_circles, im_blurred)
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))

In [ ]:
image = Image.new('L', (image_width, image_height))
draw = ImageDraw.Draw(image)

w, h = 120, 80
#draw.rectangle([20, 20, w, h], outline="white")
draw.circle([50, 50], 10, outline="white")
draw.circle([70, 30], 5, outline="white")

im = jnp.array(image)
im_blurred = gaussian_blur(im)

plt.imshow(im_blurred, origin='lower')

In [ ]:
params = jnp.array([10., 10, 5, 150, 20, 5])

In [ ]:
@jax.jit
def loss_fn_two_circles_minimum(params, nonzero_points):
    def sdf(point):
        return jnp.array([
                distance_to_circle(point[1], point[0], *params[0:3])**2,
                distance_to_circle(point[1], point[0], *params[3:6])**2,
                ]).min()
    loss = jax.vmap(sdf)(nonzero_points)
    return loss.sum()

solution = solve(params, loss_fn_two_circles_minimum, im_blurred)
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))

In [ ]:
# Second circle stuck in corner because its gradient vanishes
grad_loss = jax.grad(loss_fn_two_circles_minimum)
grad_loss(params, jnp.argwhere(im_blurred))

In [ ]:
@jax.jit
def loss_fn_one_circle_log(params, nonzero_points):
    def sdf(point):
        return jnp.log(distance_to_circle(point[1], point[0], *params[0:3])**2)
    loss = jax.vmap(sdf)(nonzero_points)
    return loss.sum()

solution = solve(params, loss_fn_one_circle_log, im_blurred)
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))

In [ ]:
def soft_min(sdfs, beta=0.001):
    exp_sdfs = jnp.exp(-beta * jnp.array(sdfs))
    return -jnp.log(jnp.sum(exp_sdfs)) / beta

@jax.jit
def loss_fn_two_circles_soft_minimum(params, nonzero_points):
    def sdf(point):
        return soft_min(jnp.array([
                distance_to_circle(point[1], point[0], *params[0:3])**2,
                distance_to_circle(point[1], point[0], *params[3:6])**2,
                ]))
    loss = jax.vmap(sdf)(nonzero_points)
    return loss.sum()

solution = solve(params, loss_fn_two_circles_soft_minimum, im_blurred)
plot_circle_on_image(im_blurred, params.reshape(-1, 3))
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))
solution

In [ ]:
known_solution = jnp.array([50., 50, 10, 70, 30, 5])
nonzero_points = jnp.argwhere(im_blurred)
(loss_fn_two_circles_soft_minimum(known_solution, nonzero_points) # 300
, loss_fn_two_circles_soft_minimum(solution, nonzero_points)) # 11,370

In [ ]:
def weighted_sum(sdfs, epsilon=0.01):
    weights = 1 / (sdfs**2 + epsilon)
    return (weights * sdfs).sum() / weights.sum()

@jax.jit
def loss_fn_two_circles_weighted_sum(params, nonzero_points):
    def sdf(point):
        return weighted_sum(jnp.array([
                distance_to_circle(point[1], point[0], *params[0:3]),
                distance_to_circle(point[1], point[0], *params[3:6]),
                ]))**2
    loss = jax.vmap(sdf)(nonzero_points)
    return loss.sum()

solution = solve(params, loss_fn_two_circles_weighted_sum, im_blurred)
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))
solution

## New loss

Let's try a new approach where we evaluate the SDFs across the entire target image, not just on the nonzero pixels.
We have to wrap the gradient function so that it doesn't turn into NaN at bad points (e.g., center of circle)

In [ ]:
import optax
import optax.tree_utils as otu

from functools import partial
_lbfgs = optax.lbfgs()


@partial(jax.jit, static_argnames=["fun"])
def _run_lbfgs(init_params, lbfgs_state, fun, max_steps, tolerance, **kwargs):
    value_and_grad_fun = optax.value_and_grad_from_state(fun)
    
    def step(carry):
        params, state = carry
        value, grad = value_and_grad_fun(params, **kwargs, state=state)

        jax.debug.print("loss: {value}, grad: {grad}, params: {params}", value=value, grad=grad, params=params)
        grad = jnp.nan_to_num(grad, nan=0.0)

        updates, state = _lbfgs.update(
            grad, state, params, **kwargs, value=value, grad=grad, value_fn=fun
        )
        params = optax.apply_updates(params, updates)

        return params, state

    def continuing_criterion(carry):
        _, state = carry
        iter_num = otu.tree_get(state, "count")
        grad = otu.tree_get(state, "grad")
        err = otu.tree_l2_norm(grad)
        return (iter_num == 0) | ((iter_num < max_steps) & (err >= tolerance))

    if lbfgs_state is None:
        state = _lbfgs.init(init_params)
    else:
        # reset iteration count leftover from previous solve
        state = otu.tree_set(lbfgs_state, count=0)


    init_carry = (init_params, state)
    final_params, final_state = jax.lax.while_loop(
        continuing_criterion, step, init_carry
    )
    return final_params, final_state


def enumerate_pixels(image):
    y, x = jnp.meshgrid(jnp.arange(image.shape[0]), jnp.arange(image.shape[1]), indexing='ij')
    y = y.flatten()
    x = x.flatten()
    values = image.flatten()
    enumerated_pixels = jnp.column_stack((x, y, values))
    return enumerated_pixels

def solve(params, loss_fn, target_image):
    enumerated_pixels = enumerate_pixels(target_image)
    start_time = time.time()
    soln, state = _run_lbfgs(
        params,
        None, # minimizer state
        loss_fn,
        max_steps=1000,
        tolerance=1e-3,
        enumerated_pixels=enumerated_pixels,

    )
    end_time = time.time()
    iter_num = otu.tree_get(state, "count")
    print(f"Elapsed time: {end_time - start_time:.5f} seconds; {iter_num} iterations")

    return soln

In [ ]:

def gaussian(x, sigma=1.0):
    return jnp.exp(-x**2 / (2 * sigma**2))

@jax.jit
def loss(params, enumerated_pixels):
    def pixel_loss(pixel):
        x, y, v = pixel
        # sm = soft_min(jnp.array([
        #         distance_to_circle(x, y, *params[0:3])**2,
        #         distance_to_circle(x, y, *params[3:6])**2,
        # ]))
        # return -v * jnp.exp(-sm)

        distances = jnp.array([
            distance_to_circle(x, y, *params[0:3]),
            distance_to_circle(x, y, *params[3:6]),
        ])
        
        # Attraction term: high when circles are far from positive pixels
        attraction = v * (1 - gaussian(distances, sigma=5.0))
        
        # Repulsion term: high when circles are close to negative pixels
        repulsion = (1 - v) * gaussian(distances, sigma=5.0)
        
        positive_radius = jnp.exp(-100*params[2]) + jnp.exp(-100*params[5])

        return jnp.sum(attraction) + jnp.sum(repulsion) + positive_radius
    
    loss = jax.vmap(pixel_loss)(enumerated_pixels)
    loss = loss.sum()
    jax.debug.print("{loss}, {params}", loss=loss, params=params)
    return loss

solution = solve(params, loss, im_blurred)
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))
solution

In [ ]:
pixels = enumerate_pixels(im_blurred)
pixels = pixels[pixels[:, 2] > 0]
jax.grad(loss)(params, pixels)

In [ ]:
(loss(known_solution, enumerate_pixels(im_blurred)),
 loss(solution, enumerate_pixels(im_blurred)))

# Let's try to sort out vanishing gradient on a small example

In [ ]:
image_width, image_height = 30, 30
image = Image.new('L', (image_width, image_height))
draw = ImageDraw.Draw(image)

true_solution = jnp.array([10., 10, 4])
draw.circle(true_solution[0:2], true_solution[2], outline="white")
im = jnp.array(image)
im_blurred = gaussian_blur(im)
im_blurred = im_blurred / im_blurred.max()
plt.imshow(im_blurred, origin='lower')

In [ ]:
@jax.jit
def loss(params, enumerated_pixels):
    def pixel_loss(pixel):
        x, y, v = pixel
        distance = distance_to_circle(x, y, *params[0:3])
        # at small distances we want intensity to be 1, and at large distances we want intensity to be 0
        expected_intensity = jnp.exp(-distance**2)
        return (expected_intensity - v)**2

    loss = jax.vmap(pixel_loss)(enumerated_pixels)
    loss = loss.sum()

    # prior on radius
    radius = params[2]
    radius_penalty = jnp.where(
        (radius >= 2) & (radius <= 10),
        0.0,
        100 * jnp.minimum((radius - 2)**2, (radius - 10)**2)
    )
    
    return loss + radius_penalty

params = jnp.array([15.1, 15.1, 0.5])

solution = solve(params, loss, im_blurred)
plot_circle_on_image(im_blurred, solution.reshape(-1, 3))

(solution,
loss(solution, enumerate_pixels(im_blurred)),
 loss(true_solution, enumerate_pixels(im_blurred)))